In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
from optbinning import OptimalBinning

# ============================================================
# 1. LOAD 3 FILE PD_CALIBRATED
# ============================================================

segment_files = [
    r'C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg1_pd_calibrated.parquet',
    r"C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg2_pd_calibrated.parquet",
    r"C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg3_pd_calibrated.parquet"
]

df_list = []

for f in segment_files:
    temp_df = pd.read_parquet(f)
    df_list.append(temp_df)

df_total = pd.concat(df_list, axis=0, ignore_index=True)

print("Loaded:", df_total.shape)


# ============================================================
# 2. TÍNH SCORE TỪ PD
# ============================================================

PDO = 20
ODDS0 = 50
SCORE0 = 600

Factor = PDO / np.log(2)
Offset = SCORE0 + Factor * np.log(ODDS0)

def pd_to_score(pd):
    pd = np.clip(pd, 1e-6, 1-1e-6)
    log_odds = np.log(pd / (1-pd))
    return Offset - Factor * log_odds

df_total["SCORE"] = pd_to_score(df_total["PD"])

df_score_output = df_total.copy()


# ============================================================
# 3. GINI CHUNG (TRAIN / OOS / OOT)
# ============================================================

def calculate_gini(y, p):
    auc = roc_auc_score(y, p)
    return 2 * auc - 1

for dtype in ["TRAIN", "OOS", "OOT"]:
    subset = df_total[df_total["DATA_TYPE"] == dtype]
    gini = calculate_gini(subset["y"], subset["PD"])
    print(f"{dtype}: GINI = {gini:.4f}")


# ============================================================
# 4. KS – CUT-OFF TRÊN TRAIN
# ============================================================

train_df = df_total[df_total["DATA_TYPE"]=="TRAIN"]

fpr, tpr, thr = roc_curve(train_df["y"], train_df["PD"])
ks = (tpr - fpr).max()
ks_cut = thr[np.argmax(tpr - fpr)]

print("\nKS =", ks, "| Cut-off =", ks_cut)


# ============================================================
# 5. OPTBINNING – FIT TRÊN TRAIN PD
# ============================================================

optb = OptimalBinning(dtype="numerical", monotonic_trend="ascending", max_n_bins=10)
optb.fit(train_df["PD"], train_df["y"])

cuts = optb.splits
print("Cutpoints:", cuts)

df_score_output["BIN"] = np.digitize(df_score_output["PD"], cuts, right=True)


# ============================================================
# 6. MAP BIN → RATING
# ============================================================

rating_labels = ["AAA","AA+","AA","AA-","A+","A","A-","BBB","BB","B"]

bin_mean_pd = df_score_output.groupby("BIN")["PD"].mean().sort_values()
bin_to_rating = {b: rating_labels[i] for i, b in enumerate(bin_mean_pd.index)}

df_score_output["RATING"] = df_score_output["BIN"].map(bin_to_rating)


# ============================================================
# 7. SUMMARY – THEO KHOẢNG ĐIỂM
# ============================================================

summary_rows = []
for rating in rating_labels:
    df_r = df_score_output[df_score_output["RATING"] == rating]
    if df_r.empty:
        summary_rows.append([rating, None, None, 0, 0, None, None])
        continue

    score_min = df_r["SCORE"].min()
    score_max = df_r["SCORE"].max()
    pd_mean = df_r["PD"].mean()
    def_rate = df_r["y"].mean()

    summary_rows.append([
        rating,
        f"{score_min:.2f} → {score_max:.2f}",
        len(df_r),
        len(df_r) / len(df_score_output),
        pd_mean,
        def_rate
    ])

df_rating_summary = pd.DataFrame(summary_rows, columns=[
    "RATING", "SCORE_RANGE", "COUNT", "PCT", "MEAN_PD", "DEFAULT_RATE"
])

print(df_rating_summary)


# ============================================================
# 8. EXPORT MASTER FILE
# ============================================================

df_score_output.to_parquet("FINAL_MODEL_SCORED_ALL.parquet", index=False)
print("\n✔ Xuất file FINAL_MODEL_SCORED_ALL.parquet thành công!")

Loaded: (1740237, 5)
TRAIN: GINI = 0.6676
OOS: GINI = 0.6858
OOT: GINI = 0.6850

KS = 0.516161998500056 | Cut-off = 0.07725416741398945
Cutpoints: [0.00195221 0.00361638 0.00902699 0.02013405 0.03356657 0.06590865
 0.12449665 0.18416011 0.27048425]
  RATING      SCORE_RANGE   COUNT       PCT   MEAN_PD  DEFAULT_RATE
0    AAA  892.84 → 938.79  146583  0.084232  0.001123      0.001044
1    AA+  875.00 → 892.83  108375  0.062276  0.002837      0.003091
2     AA  848.45 → 875.00  242294  0.139230  0.005905      0.006199
3    AA-  824.97 → 848.44  246007  0.141364  0.014125      0.013975
4     A+  809.83 → 824.97  178195  0.102397  0.026183      0.025741
5      A  789.38 → 809.83  211833  0.121727  0.048107      0.047434
6     A-  769.16 → 789.38  256994  0.147678  0.093657      0.092940
7    BBB  755.82 → 769.16  162797  0.093549  0.151477      0.152914
8     BB  741.51 → 755.82  107816  0.061955  0.220227      0.214950
9      B  677.17 → 741.50   79343  0.045593  0.375097      0.375723

✔ 

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp
from sklearn.metrics import brier_score_loss

# ============================================================
# 1. HÀM TÍNH TOÁN CÁC CHỈ SỐ NÂNG CAO (KS, BRIER, CAPTURE RATE)
# ============================================================
def calculate_advanced_metrics_from_df(df, target_col='y', prob_col='PD', label=""):
    # Lấy dữ liệu
    y_true = df[target_col].values
    y_prob = df[prob_col].values
    
    # --- A. Tính KS Statistic (Trên tập OOT) ---
    # KS đo độ tách biệt giữa Good và Bad
    prob_good = y_prob[y_true == 0]
    prob_bad  = y_prob[y_true == 1]
    ks_stat, p_value = ks_2samp(prob_good, prob_bad)
    
    # --- B. Tính Brier Score ---
    # Đo độ chính xác của xác suất (càng thấp càng tốt)
    brier = brier_score_loss(y_true, y_prob)
    
    # --- C. Tính Bad Capture Rate (Lift) ---
    # Tỷ lệ nợ xấu bắt được trong nhóm rủi ro nhất (Top 10%, 20%)
    df_temp = df.copy()
    df_temp = df_temp.sort_values(by=prob_col, ascending=False) # Sort PD giảm dần (Rủi ro cao lên đầu)
    
    total_bads = df_temp[target_col].sum()
    n_rows = len(df_temp)
    
    # Top 10%
    top_10_idx = int(n_rows * 0.1)
    bads_in_top_10 = df_temp.iloc[:top_10_idx][target_col].sum()
    capture_rate_10 = bads_in_top_10 / total_bads if total_bads > 0 else 0
    
    # Top 20%
    top_20_idx = int(n_rows * 0.2)
    bads_in_top_20 = df_temp.iloc[:top_20_idx][target_col].sum()
    capture_rate_20 = bads_in_top_20 / total_bads if total_bads > 0 else 0
    
    print(f">>> REPORT FOR: {label}")
    print(f"   • KS Statistic:       {ks_stat:.4f} ({(ks_stat*100):.2f}%)")
    print(f"   • Brier Score:        {brier:.6f}")
    print(f"   • Capture Rate @ 10%: {capture_rate_10:.2%}")
    print(f"   • Capture Rate @ 20%: {capture_rate_20:.2%}")
    print("-" * 60)

# ============================================================
# 2. HÀM TÍNH PSI (ĐỘ ỔN ĐỊNH - TRAIN vs OOT)
# ============================================================
def calculate_psi_score(expected, actual, buckets=10):
    # expected: Score/PD tập Train
    # actual: Score/PD tập OOT
    
    # Chia bin dựa trên phân phối của tập Train (Expected)
    breakpoints = np.arange(0, buckets + 1) / (buckets) * 100
    breakpoints = np.percentile(expected, breakpoints)
    
    # Tránh lỗi trùng bin
    breakpoints = np.unique(breakpoints)
    
    # Tính % tỷ trọng
    expected_percents = np.histogram(expected, bins=breakpoints)[0] / len(expected)
    actual_percents = np.histogram(actual, bins=breakpoints)[0] / len(actual)
    
    # Xử lý division by zero (thêm một lượng cực nhỏ)
    expected_percents = np.where(expected_percents == 0, 0.0001, expected_percents)
    actual_percents = np.where(actual_percents == 0, 0.0001, actual_percents)
    
    # Công thức PSI
    psi_value = np.sum((expected_percents - actual_percents) * np.log(expected_percents / actual_percents))
    
    return psi_value

# ============================================================
# 3. THỰC HIỆN TÍNH TOÁN
# ============================================================
print("\n" + "="*40)
print("📊 BÁO CÁO CHỈ SỐ NÂNG CAO - LOGISTIC REGRESSION")
print("="*40)

# 1. Lọc dữ liệu OOT từ bảng tổng
# Lưu ý: df_score_output là bảng bạn đã tạo ở các bước trước trong file ipynb
df_oot = df_score_output[df_score_output["DATA_TYPE"] == "OOT"]
df_train = df_score_output[df_score_output["DATA_TYPE"] == "TRAIN"]

# 2. Tính các chỉ số hiệu suất trên OOT
if not df_oot.empty:
    calculate_advanced_metrics_from_df(df_oot, target_col='y', prob_col='PD', label="OOT DATASET")
else:
    print("⚠️ Không tìm thấy dữ liệu OOT!")

# 3. Tính PSI (Độ ổn định của mô hình giữa Train và OOT)
# Dùng PD để tính PSI
psi_value = calculate_psi_score(df_train["PD"], df_oot["PD"])

print(f">>> STABILITY (PSI - Train vs OOT): {psi_value:.4f}")
if psi_value < 0.1:
    print("   ✅ PSI < 0.1: Mô hình rất ổn định.")
elif psi_value < 0.25:
    print("   ⚠️ PSI 0.1-0.25: Mô hình biến động nhẹ.")
else:
    print("   ❌ PSI > 0.25: Mô hình không ổn định.")


📊 BÁO CÁO CHỈ SỐ NÂNG CAO - LOGISTIC REGRESSION
>>> REPORT FOR: OOT DATASET
   • KS Statistic:       0.5379 (53.79%)
   • Brier Score:        0.049326
   • Capture Rate @ 10%: 43.27%
   • Capture Rate @ 20%: 66.08%
------------------------------------------------------------
>>> STABILITY (PSI - Train vs OOT): 0.0308
   ✅ PSI < 0.1: Mô hình rất ổn định.


In [3]:
# # ============================================================
# # BƯỚC 11 — KS STATISTIC (TIM CUT-OFF)
# # ============================================================
# from sklearn.metrics import roc_curve
# from optbinning import OptimalBinning

# print("\n[BƯỚC 11] Tìm cut-off bằng KS...")

# fpr, tpr, thresholds = roc_curve(y_train_full, pd_train)
# ks_values = tpr - fpr

# ks_max = ks_values.max()
# ks_cutoff = thresholds[np.argmax(ks_values)]

# print(f" → KS max = {ks_max:.4f}")
# print(f" → Cut-off tối ưu = {ks_cutoff:.4f}")


# # ============================================================
# # BƯỚC 12 — XÂY THANG HẠNG (OPT BINNING)
# # ============================================================
# print("\n[BƯỚC 12] Xây thang hạng bằng OptBinning...")

# optb = OptimalBinning(
#     name="RatingScale",
#     dtype="numerical",
#     monotonic_trend="ascending",   # PD tăng → rủi ro tăng
#     max_n_bins=10
# )

# # Fit trên TRAIN
# optb.fit(pd_train, y_train_full)

# bin_table = optb.binning_table.build()
# print(bin_table)


# # ============================================================
# # BƯỚC 13 — LẤY CUTPOINTS VÀ GÁN BIN
# # ============================================================
# print("\n[BƯỚC 13] Áp dụng rating cho toàn bộ dataset...")

# cuts = optb.splits
# print("Cutpoints:", cuts)

# df_score_output["BIN"] = np.digitize(df_score_output["PD"], cuts, right=True)


# # ============================================================
# # BƯỚC 14 — GÁN 10 HẠNG (AAA → CC)
# # ============================================================
# print("\n[BƯỚC 14] Mapping BIN → RATING...")

# bin_mean_pd = df_score_output.groupby("BIN")["PD"].mean().sort_values()


# rating_labels = ["AAA","AA+","AA","AA-","A+","A","A-","BBB","BB","B"]

# sorted_bins = bin_mean_pd.index.tolist()

# bin_to_rating = {b: rating_labels[i] for i, b in enumerate(sorted_bins)}

# df_score_output["RATING"] = df_score_output["BIN"].map(bin_to_rating)


# print(df_score_output[["SEGMENT", "PD", "SCORE", "BIN", "RATING"]].head())


# # ============================================================
# # BƯỚC 15 — KIỂM TRA ĐƠN ĐIỆU (MONOTONICITY)
# # ============================================================
# print("\n[BƯỚC 15] Kiểm tra tính đơn điệu của PD theo Rating...")

# rating_pd = df_score_output.groupby("RATING")["PD"].mean().reindex(rating_labels)
# print(rating_pd)

# is_monotonic = rating_pd.is_monotonic_increasing
# print(" → PD có đơn điệu không? →", is_monotonic)


# # ============================================================
# # BƯỚC 16 — KIỂM TRA SỰ KHÁC BIỆT RỦI RO GIỮA CÁC HẠNG
# # ============================================================
# print("\n[BƯỚC 16] Kiểm tra độ tách bạch giữa các rating...")

# rating_diff = rating_pd.diff()
# print(rating_diff)


# # ============================================================
# # BƯỚC 17 — KIỂM TRA MỨC ĐỘ TẬP TRUNG
# # ============================================================
# print("\n[BƯỚC 17] Kiểm tra mức độ tập trung khách hàng...")

# rating_dist = df_score_output["RATING"].value_counts(normalize=True)
# print(rating_dist)


# # ============================================================
# # BƯỚC 18 — TÍNH HHI
# # ============================================================
# print("\n[BƯỚC 18] Tính Herfindahl-Hirschman Index (HHI)...")

# HHI = np.sum(rating_dist ** 2)
# print(f" → HHI = {HHI:.6f}")

# if HHI < 0.10:
#     print(" → Phân bố rating RẤT TỐT (rất đều).")
# elif HHI < 0.18:
#     print(" → Phân bố rating CHẤP NHẬN ĐƯỢC.")
# else:
#     print(" → Tập trung QUÁ MỨC → nên xem lại cách binning.")


# # ============================================================
# # BƯỚC 19 — XUẤT BẢNG CUỐI
# # ============================================================
# print("\n[BƯỚC 19] Xuất bảng cuối cùng...")

# df_final_rating = df_score_output.copy()
# display(df_final_rating.head())
    
# print("→ Hoàn tất toàn bộ Rating Pipeline!")


In [4]:
# # ============================================================
# # BẢNG THANG HẠNG CUỐI — SUMMARY TABLE (THEO KHOẢNG ĐIỂM)
# # ============================================================

# print("\n[BẢNG TỔNG HỢP THANG HẠNG — KHOẢNG ĐIỂM]")

# summary_list = []

# for stt, rating in enumerate(rating_labels, start=1):

#     df_r = df_score_output[df_score_output["RATING"] == rating]

#     if df_r.empty:
#         summary_list.append([
#             stt,
#             rating,
#             "",
#             0,
#             0,
#             None,
#             None
#         ])
#         continue

#     # 🔹 KHOẢNG ĐIỂM (SCORE RANGE)
#     score_min = df_r["SCORE"].min()
#     score_max = df_r["SCORE"].max()
#     score_range = f"{score_min:.2f} → {score_max:.2f}"

#     # Số lượng
#     count = len(df_r)
#     pct = count / len(df_score_output)

#     # PD ước tính trung bình
#     pd_est = df_r["PD"].mean()

#     # Tỷ lệ vỡ nợ thực tế
#     default_rate = df_r["y"].mean() if "y" in df_r.columns else None

#     summary_list.append([
#         stt,
#         rating,
#         score_range,
#         count,
#         pct,
#         pd_est,
#         default_rate
#     ])

# # Tạo DataFrame
# df_rating_summary = pd.DataFrame(summary_list, columns=[
#     "STT",
#     "Hạng",
#     "Khoảng điểm (Score)",
#     "Số lượng quan sát",
#     "Số lượng quan sát (%)",
#     "PD ước tính trung bình",
#     "Tỷ lệ vỡ nợ thực tế"
# ])

# # Format
# df_rating_summary["Số lượng quan sát (%)"] = df_rating_summary["Số lượng quan sát (%)"].apply(lambda x: f"{x:.2%}")
# df_rating_summary["PD ước tính trung bình"] = df_rating_summary["PD ước tính trung bình"].apply(lambda x: None if pd.isna(x) else round(x, 6))
# df_rating_summary["Tỷ lệ vỡ nợ thực tế"] = df_rating_summary["Tỷ lệ vỡ nợ thực tế"].apply(lambda x: None if pd.isna(x) else round(x, 6))

# display(df_rating_summary)

# print("\n→ Đã dựng xong bảng thang hạng cuối cùng (khoảng điểm).")
